
# Appraisal Scope — Reorganized Analysis Notebook

This notebook has been reorganized into a clean, reproducible flow:

1) **Setup & Config**  
2) **Extract Data** (from GBQ/CSV/SQL/etc.)  
3) **Combine DataFrames** (joins, alignment, target/feature selection)  
4) **Feature Engineering** (lags, rolling stats, seasonal flags, YoY/MoM)  
5) **Visual EDA** (time series charts, *dual‑axis* helper across all variables)  
6) **Statistical Tests** (correlation, ADF, ACF/PACF, normality, Granger causality)  
7) **Models** (OLS, SARIMAX, Weighted Lag, XGBoost, Naive 12‑lag + 3‑mo YoY growth)  
8) **Evaluation & Model Selection** (RMSE/MAE/MAPE/SMAPE, backtests)  
9) **Save Artifacts** (CSVs, charts, model summaries)

> Notes:  
- Update paths and variable names as needed.  
- XGBoost import is optional (gracefully skipped if unavailable).  
- Keep target column named `y` for simplicity (rename in Section 3).


## 1) Setup & Config

In [ ]:

# Standard
import os, sys, math, warnings
warnings.filterwarnings("ignore")

# Data + Stats
import numpy as np
import pandas as pd

# Plots
import matplotlib.pyplot as plt

# Time series & stats tests
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.stats.diagnostic import acorr_ljungbox, het_breuschpagan, normal_ad
from statsmodels.stats.stattools import jarque_bera

# Machine learning
try:
    from xgboost import XGBRegressor
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

# Display options
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

# Random seed for reproducibility
RANDOM_STATE = 42

# Helper: metrics
def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    denom = np.where(denom == 0, 1.0, denom)
    return np.mean(np.abs(y_pred - y_true) / denom) * 100

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = np.where(y_true == 0, 1.0, y_true)
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

def mae(y_true, y_pred):
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

def rmse(y_true, y_pred):
    return math.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


## 2) Extract Data

In [ ]:

# === Replace this block with your real extractor (GBQ/SQL/API). ===
# For now, read from CSVs as an example.
# Expected: a time column `date` (monthly) and at least one target column (e.g., 'total_cases_opened').
# Example:
# df_target = pd.read_csv('/mnt/data/appscope_target.csv', parse_dates=['date'])
# df_drivers = pd.read_csv('/mnt/data/appscope_drivers.csv', parse_dates=['date'])

df_target = pd.DataFrame({
    'date': pd.date_range('2022-01-01', periods=36, freq='MS'),
    'total_cases_opened': (np.random.rand(36)*1000 + 3000).round(0)
})
df_drivers = pd.DataFrame({
    'date': pd.date_range('2022-01-01', periods=36, freq='MS'),
    'UNRATE': np.random.rand(36)*5 + 3,
    'MORTGAGE30US': np.random.rand(36)*2 + 5,
    'orders': (np.random.rand(36)*12000+40000).round(0)
})
df_target.head(), df_drivers.head()


## 3) Combine DataFrames

In [ ]:

# Align by month start and set the target as `y`
df = pd.merge(df_target, df_drivers, on='date', how='inner').sort_values('date')
df = df.set_index('date')
df['y'] = df['total_cases_opened']
df = df.drop(columns=['total_cases_opened'])

print("Combined shape:", df.shape)
display(df.tail())


## 4) Feature Engineering

In [ ]:

# Lags (12), rolling means, seasonal dummies, YoY growth (3-month YoY variant)
for lag in range(1, 13):
    df[f'y_lag{lag}'] = df['y'].shift(lag)

# Driver lags
for col in [c for c in df.columns if c not in ['y'] and not c.startswith('y_lag')]:
    for lag in [1,3,6,12]:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)

# Rolling features on y
df['y_roll3'] = df['y'].rolling(3).mean()
df['y_roll6'] = df['y'].rolling(6).mean()
df['y_roll12'] = df['y'].rolling(12).mean()

# Seasonality
df['month'] = df.index.month
df = pd.get_dummies(df, columns=['month'], prefix='m', drop_first=True)

# 3-month YoY growth: (y_t - y_{t-12}) / y_{t-12} averaged over last 3 months
df['y_yoy'] = (df['y'] - df['y'].shift(12)) / df['y'].shift(12)
df['y_yoy_3m'] = df['y_yoy'].rolling(3).mean()

display(df.tail(15))


## 5) Visual EDA — Time Series & Dual‑Axis Helper

In [ ]:

# Helper to dual-axis plot any driver vs target y
def plot_dual_axis(df, driver_col, target_col='y', title=None):
    if title is None:
        title = f"{driver_col} vs {target_col}"
    ax = df[[target_col]].plot(figsize=(10,4))
    df[[driver_col]].plot(ax=ax, secondary_y=True)
    ax.set_title(title)
    ax.grid(True)
    plt.show()

# Plot y
df[['y']].plot(figsize=(10,4), title='Target y'); plt.grid(True); plt.show()

# Plot all drivers (non-lag, non-dummy) against y as dual-axis
base_cols = [c for c in df.columns if not c.startswith('y_lag') and not c.startswith('m_')
             and c not in ['y','y_roll3','y_roll6','y_roll12','y_yoy','y_yoy_3m']]
for col in base_cols:
    plot_dual_axis(df, col, target_col='y', title=f"{col} vs y")


## 6) Statistical Tests

In [ ]:

from itertools import combinations
import statsmodels.tsa.stattools as tsast
import numpy as np
import pandas as pd

# Correlation matrix (Pearson) for base columns + y
corr_cols = ['y'] + base_cols
corr = df[corr_cols].dropna().corr(method='pearson')
display(corr)

# ADF test on y
adf_res = adfuller(df['y'].dropna(), autolag='AIC')
print("ADF Statistic:", adf_res[0])
print("p-value:", adf_res[1])
print("Critical Values:", adf_res[4])

# Autocorrelation and partial autocorrelation
series = df['y'].dropna()
lags_to_show = min(24, len(series)-1) if len(series) > 1 else 1
acf_vals = acf(series, nlags=lags_to_show, fft=True)
pacf_vals = pacf(series, nlags=lags_to_show, method='ywunbiased')
print("ACF (first 10):", np.round(acf_vals[:10],3))
print("PACF (first 10):", np.round(pacf_vals[:10],3))

# Normality tests
jb_stat, jb_p, _, _ = jarque_bera(series)
print(f"Jarque-Bera: stat={jb_stat:.3f}, p={jb_p:.5f}")
ad_stat, ad_p = normal_ad(series)
print(f"Anderson-Darling normality (approx): stat={ad_stat:.3f}, p={ad_p:.5f}")

# Granger causality tests (each driver -> y), maxlag up to 12 where feasible
maxlag = max(1, min(12, len(df.dropna())//4))
granger_results = {}
for col in base_cols:
    try:
        out = tsast.grangercausalitytests(df[[ 'y', col ]].dropna(), maxlag=maxlag, verbose=False)
        # collect min p-value across lags for F-test
        pvals = [out[lag][0]['ssr_ftest'][1] for lag in out.keys()]
        granger_results[col] = float(np.min(pvals))
    except Exception:
        granger_results[col] = np.nan

granger_df = pd.DataFrame({'driver': list(granger_results.keys()), 'min_pvalue': list(granger_results.values())}).sort_values('min_pvalue')
display(granger_df)


## 7) Modeling — OLS, SARIMAX, Weighted Lag, XGB, Naive(12‑lag)+YoY

In [ ]:

# Train/test split (last 6 months as test by default)
horizon = 6
data = df.dropna().copy()
if len(data) < horizon + 13:
    horizon = max(1, min(3, len(data)//4))

train, test = data.iloc[:-horizon], data.iloc[-horizon:]

# Feature set for ML/OLS (avoid leakage: exclude direct future info)
exclude_prefixes = ('m_',)
exclude_exact = set(['y','y_roll3','y_roll6','y_roll12','y_yoy'])  # keep y_yoy_3m if desired
X_cols = [c for c in train.columns if c not in exclude_exact and not any(c.startswith(p) for p in exclude_prefixes)]
y_col = 'y'

X_train, y_train = train[X_cols], train[y_col]
X_test, y_test = test[X_cols], test[y_col]

print(f"Features ({len(X_cols)}):", X_cols[:12], '...')

# 7.1 OLS
X_train_c = sm.add_constant(X_train, has_constant='add')
ols_model = sm.OLS(y_train, X_train_c).fit()
print(ols_model.summary())

# 7.2 SARIMAX (simple seasonal guess — adjust as needed)
try:
    sarimax_mod = tsa.SARIMAX(train[y_col], order=(1,1,1), seasonal_order=(1,1,1,12),
                              enforce_stationarity=False, enforce_invertibility=False)
    sarimax_fit = sarimax_mod.fit(disp=False)
    sarimax_forecast = sarimax_fit.forecast(steps=len(test))
except Exception:
    sarimax_fit, sarimax_forecast = None, pd.Series(index=y_test.index, dtype=float)

# 7.3 Weighted Lag model (example: blend of recent lags)
def weighted_lag_predict(train_series, horizon, weights=None):
    if weights is None:
        # Example: heavier weight on recent months: lags 1,2,3,12
        weights = {1:0.45, 2:0.25, 3:0.15, 12:0.15}
    preds = []
    hist = train_series.copy().tolist()
    for _ in range(horizon):
        val = 0.0
        for lag, w in weights.items():
            if len(hist) - lag >= 0:
                val += hist[-lag] * w
        preds.append(val)
        hist.append(val)
    return pd.Series(preds, index=test.index)

weighted_preds = weighted_lag_predict(train[y_col], horizon=len(test))

# 7.4 XGBoost (optional if installed)
if XGB_AVAILABLE and len(X_train) > 0 and len(X_test) > 0:
    xgb = XGBRegressor(
        n_estimators=300, learning_rate=0.05, max_depth=4,
        subsample=0.9, colsample_bytree=0.9, random_state=RANDOM_STATE
    )
    xgb.fit(X_train, y_train)
    xgb_preds = pd.Series(xgb.predict(X_test), index=y_test.index)
else:
    xgb_preds = pd.Series(index=y_test.index, dtype=float)

# 7.5 Naive 12‑lag + 3‑month YoY growth
def naive_12lag_yoy3(train_series, horizon):
    preds = []
    yoy = (train_series - train_series.shift(12)) / train_series.shift(12)
    yoy3 = yoy.rolling(3).mean().iloc[-1]
    yoy3 = 0.0 if pd.isna(yoy3) else float(yoy3)
    hist = train_series.copy()
    for _ in range(horizon):
        if len(hist) >= 12:
            base = hist.iloc[-12]
        else:
            base = hist.iloc[-1]
        pred = base * (1 + yoy3)
        preds.append(pred)
        hist = pd.concat([hist, pd.Series([pred])])
    return pd.Series(preds, index=test.index)

naive_preds = naive_12lag_yoy3(train[y_col], horizon=len(test))

# Predictions for OLS and SARIMAX
ols_preds = pd.Series(ols_model.predict(sm.add_constant(X_test, has_constant='add')), index=y_test.index)
sarimax_preds = sarimax_forecast if sarimax_fit is not None else pd.Series(index=y_test.index, dtype=float)

preds_df = pd.DataFrame({
    'y_true': y_test,
    'OLS': ols_preds,
    'SARIMAX': sarimax_preds,
    'WeightedLag': weighted_preds,
    'XGB' : xgb_preds,
    'Naive12LagYoY3': naive_preds
})
display(preds_df)


## 8) Evaluation & Model Selection

In [ ]:

def eval_all(y_true, pred_dict):
    import numpy as np, math
    def smape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
        denom = np.where(denom == 0, 1.0, denom)
        return np.mean(np.abs(y_pred - y_true) / denom) * 100
    def mape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        denom = np.where(y_true == 0, 1.0, y_true)
        return np.mean(np.abs((y_true - y_pred) / denom)) * 100
    def mae(y_true, y_pred):
        return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))
    def rmse(y_true, y_pred):
        return math.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))
    rows = []
    for name, yhat in pred_dict.items():
        if len(yhat.dropna()) == 0:
            continue
        rows.append({
            'model': name,
            'RMSE': rmse(y_true, yhat),
            'MAE': mae(y_true, yhat),
            'MAPE': mape(y_true, yhat),
            'SMAPE': smape(y_true, yhat)
        })
    return pd.DataFrame(rows).sort_values('SMAPE')

metrics = eval_all(preds_df['y_true'], {c: preds_df[c].dropna() for c in preds_df.columns if c != 'y_true'})
display(metrics)

# Plot forecasts vs actuals
ax = preds_df[['y_true']].plot(figsize=(10,4), title='Forecast vs Actuals')
for c in preds_df.columns:
    if c != 'y_true':
        preds_df[[c]].plot(ax=ax)
ax.grid(True); plt.show()


## 9) Save Artifacts

In [ ]:

out_dir = "/mnt/data/appscope_outputs"
os.makedirs(out_dir, exist_ok=True)

preds_df.to_csv(os.path.join(out_dir, "predictions.csv"), index=True)
metrics.to_csv(os.path.join(out_dir, "metrics.csv"), index=False)

print("Saved:", os.listdir(out_dir))


---
# Appendix — Original Notebook (verbatim)

**(Original) Code Cell:**

In [ ]:
import os, sys, warnings, pytz, json
import pandas as pd
import numpy as np


warnings.filterwarnings("ignore")  # keep logs tidy

# ------------------- helper-package path --------------------
sys.path.append(r"C:\WFM_Scripting\Automation")  # adjust if needed
from scripthelper import Config, Logger, BigQueryManager, EmailManager
# ==============================================================================
# SECTION 1: DATA SOURCES (QUERIES AND FILE PATHS)
# ==============================================================================
# Define all queries and file paths
sql_tickets_historicals = r"C:\WFM_Scripting\Forecasting\GBQ - Non-Tax Platform Ticket Timeseries by Month.sql"
sql_workload_driver = "SELECT MonthOfOrder, client_id, TotalOrders FROM tax_clnt_svcs.view_cx_nontax_platforms_workload_drivers"
sql_fred = "SELECT Date, UNRATE, HSN1F, FEDFUNDS, MORTGAGE30US FROM tax_clnt_svcs.fred WHERE Date >= '2023-01-01'"
file_market_guidance = r"C:\Users\jhgonzalez\OneDrive - CoreLogic Solutions, LLC\Desktop\Market_G.xlsx"

# ==============================================================================
# SECTION 2: DATA LOADING
# ==============================================================================
# Initialize the connection manager and load all data sources
print("Loading data from sources...")
bq_manager = BigQueryManager(Config(rpt_id=9999)) # Use your report ID

tickets_historicals = bq_manager.run_gbq_sql(sql_tickets_historicals, return_dataframe=True)
workload_driver = bq_manager.run_gbq_sql(sql_workload_driver, return_dataframe=True)
FRED = bq_manager.run_gbq_sql(sql_fred, return_dataframe=True)
market_guidance = pd.read_excel(file_market_guidance)

print("All data sources loaded successfully.")

# ==============================================================================
# SECTION 3: MERGING ALL DATA SOURCES
# ==============================================================================
print("\nMerging all data sources...")

# Prepare workload_driver with special client mappings
workload_driver_prep = workload_driver.copy()
client_mapping = {
    'FNC - CMS': 'FNC',
    'FNC - Ports': 'FNC',
    'Mercury Integrations': 'Mercury'
}
workload_driver_prep['client_id'] = workload_driver_prep['client_id'].replace(client_mapping)
workload_driver_agg = workload_driver_prep.groupby(['MonthOfOrder', 'client_id'])['TotalOrders'].sum().reset_index()
workload_driver_agg = workload_driver_agg.rename(columns={'MonthOfOrder': 'date'})
workload_driver_agg['date'] = pd.to_datetime(workload_driver_agg['date'])

# Prepare tickets_historicals with a temporary join key for mapping
tickets_historicals['date'] = pd.to_datetime(tickets_historicals['date'])
tickets_historicals_prep = tickets_historicals.copy()
tickets_historicals_prep['join_key'] = tickets_historicals_prep['client_id']
fnc_variants = ['FNC - CMS', 'FNC - Ports']
tickets_historicals_prep.loc[tickets_historicals_prep['client_id'].isin(fnc_variants), 'join_key'] = 'FNC'
mercury_start_date = pd.to_datetime('2023-05-01')
mercury_mask = (tickets_historicals_prep['client_id'] == 'Mercury Integrations') & (tickets_historicals_prep['date'] >= mercury_start_date)
tickets_historicals_prep.loc[mercury_mask, 'join_key'] = 'Mercury'

# Merge tickets with workload
merged_df = pd.merge(
    tickets_historicals_prep,
    workload_driver_agg,
    left_on=['date', 'join_key'],
    right_on=['date', 'client_id'],
    how='left'
)
merged_df = merged_df.drop(columns=['join_key', 'client_id_y']).rename(columns={'client_id_x': 'client_id'})

# Merge with FRED data
FRED['Date'] = pd.to_datetime(FRED['Date'])
FRED_prep = FRED.rename(columns={'Date': 'date'})
final_df = pd.merge(merged_df, FRED_prep, on='date', how='left')

# Merge with Market Guidance data
market_guidance['DATE'] = pd.to_datetime(market_guidance['DATE'])
market_guidance_prep = market_guidance.rename(columns={'DATE': 'date'})
final_df = pd.merge(final_df, market_guidance_prep, on='date', how='left')
print("All merges complete.")


df = final_df
# ==============================================================================
# SECTION 4: DATA CLEANING (IMPUTATION)
# ==============================================================================
print("\nCleaning final DataFrame by filling missing values...")
columns_to_fill = [
    'UNRATE', 'HSN1F', 'FEDFUNDS', 'MORTGAGE30US',
    'Purchase', 'Refinance', 'Year-over-year Variance'
]
# Forward fill to propagate last known values
for col in columns_to_fill:
    final_df[col] = final_df[col].ffill()
# Backward fill to handle any NaNs at the very start
for col in columns_to_fill:
    final_df[col] = final_df[col].bfill()
print("Missing values filled.")

# ==============================================================================
# SECTION 5: SAVE FINAL, CLEANED DATA
# ==============================================================================
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')
output_file_path = os.path.join(desktop_path, 'merged_forecasting_data.csv')

try:
    final_df.to_csv(output_file_path, index=False)
    print(f"\n✅ DataFrame with all data successfully saved to:\n{output_file_path}")
except Exception as e:
    print(f"\n🔥 An error occurred while saving the file: {e}")

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# --- 1. Prepare the Data ---
# Ensure 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter for the 'Appraisal Scope' client
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()


# --- 2. Create the Plot ---
plt.figure(figsize=(12, 6))
plt.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], marker='o', linestyle='-')

# Formatting the x-axis for readability
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45)

# --- 3. Final Touches ---
plt.title('Time Series of Total Cases Opened for Appraisal Scope')
plt.xlabel('Date')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.tight_layout()
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Prepare the Data ---
# Ensure 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter for the 'Appraisal Scope' client
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()


# --- 2. Create the Plot ---
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plotting the primary variable (Total Cases Opened)
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True) # Add grid lines for the primary axis

# Creating the second y-axis that shares the same x-axis
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Business Day Count', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['business_day_count'], color=color, linestyle='--', label='Business Day Count')
ax2.tick_params(axis='y', labelcolor=color)

# Manually set the y-axis limits to ensure variation is visible
min_val = appraisal_scope_df['business_day_count'].min()
max_val = appraisal_scope_df['business_day_count'].max()
ax2.set_ylim(min_val - 1, max_val + 1)

# --- 3. Final Touches ---
plt.title('Total Cases Opened vs. Business Day Count for Appraisal Scope')
fig.tight_layout()
# Combine legends from both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- 1. Prepare the Data ---
# Ensure 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter for the 'Appraisal Scope' client
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()


# --- 2. Create the Plot ---
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plotting the primary variable (Total Cases Opened)
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

# Creating the second y-axis that shares the same x-axis
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Total Orders', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['TotalOrders'], color=color, linestyle='--', label='Total Orders')
ax2.tick_params(axis='y', labelcolor=color)


# --- 3. Final Touches ---
plt.title('Total Cases Opened vs. Total Orders for Appraisal Scope')
fig.tight_layout()
# Combine legends from both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('UNRATE (%)', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['UNRATE'], color=color, linestyle='--', label='UNRATE')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. Unemployment Rate (UNRATE)')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('New Houses Sold (HSN1F)', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['HSN1F'], color=color, linestyle='--', label='HSN1F')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. New Houses Sold (HSN1F)')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:olive'
ax2.set_ylabel('Federal Funds Rate (%)', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['FEDFUNDS'], color=color, linestyle='--', label='FEDFUNDS')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. Federal Funds Rate')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('30-Year Mortgage Rate (%)', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['MORTGAGE30US'], color=color, linestyle='--', label='MORTGAGE30US')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. 30-Year Mortgage Rate')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:brown'
ax2.set_ylabel('Purchase Market Guidance', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['Purchase'], color=color, linestyle='--', label='Purchase Guidance')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. Purchase Market Guidance')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Refinance Market Guidance', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['Refinance'], color=color, linestyle='--', label='Refinance Guidance')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. Refinance Market Guidance')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Cases Opened', color=color)
ax1.plot(appraisal_scope_df['date'], appraisal_scope_df['total_cases_opened'], color=color, marker='o', label='Total Cases Opened')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Year-over-year Variance', color=color)
ax2.plot(appraisal_scope_df['date'], appraisal_scope_df['Year-over-year Variance'], color=color, linestyle='--', label='YoY Variance')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total Cases Opened vs. Year-over-year Variance')
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. Load and Prepare Data ---
# Load the dataset from your local file
df = pd.read_csv(r'C:\Users\jhgonzalez\Desktop\merged_forecasting_data.csv')

# Filter for the 'Appraisal Scope' client
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()

# --- 2. Clean Data Types (The Fix) ---
# Convert the columns to a numeric data type.
# errors='coerce' will turn any non-numeric values into NaN (Not a Number).
appraisal_scope_df['TotalOrders'] = pd.to_numeric(appraisal_scope_df['TotalOrders'], errors='coerce')
appraisal_scope_df['total_cases_opened'] = pd.to_numeric(appraisal_scope_df['total_cases_opened'], errors='coerce')

# Drop rows where either of the columns has NaN values after conversion
appraisal_scope_df.dropna(subset=['TotalOrders', 'total_cases_opened'], inplace=True)

# --- 3. Create the Plot ---
# This plotting code will now work correctly with the cleaned data
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='TotalOrders', y='total_cases_opened')
plt.title('Total Cases Opened vs. Total Orders for Appraisal Scope')
plt.xlabel('Total Orders')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='UNRATE', y='total_cases_opened')
plt.title('Total Cases Opened vs. Unemployment Rate (UNRATE)')
plt.xlabel('UNRATE (%)')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:

plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='HSN1F', y='total_cases_opened')
plt.title('Total Cases Opened vs. New Houses Sold (HSN1F)')
plt.xlabel('New Houses Sold (in Thousands)')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='HSN1F', y='total_cases_opened')
plt.title('Total Cases Opened vs. New Houses Sold (HSN1F)')
plt.xlabel('New Houses Sold (in Thousands)')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='FEDFUNDS', y='total_cases_opened')
plt.title('Total Cases Opened vs. Federal Funds Rate')
plt.xlabel('Federal Funds Rate (%)')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='Purchase', y='total_cases_opened')
plt.title('Total Cases Opened vs. Purchase Market Guidance')
plt.xlabel('Purchase Market Guidance')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=appraisal_scope_df, x='Year-over-year Variance', y='total_cases_opened')
plt.title('Total Cases Opened vs. Year-over-year Variance')
plt.xlabel('Year-over-year Variance')
plt.ylabel('Total Cases Opened')
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# This script assumes 'appraisal_scope_df' is your prepared DataFrame.

# --- 1. Select only numeric columns for the correlation matrix ---
numeric_df = appraisal_scope_df.select_dtypes(include=['float64', 'int64'])

# --- 2. Calculate and print the correlation with the target variable ---
# This gives you a direct, sorted list of the most important relationships.
target_correlations = numeric_df.corr()['total_cases_opened'].sort_values(ascending=False)
print("--- Correlation with 'total_cases_opened' ---")
print(target_correlations)

# --- 3. Generate the correlation heatmap (table) ---
# This provides a full view of how all variables relate to each other.
plt.figure(figsize=(14, 10))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap for Appraisal Scope Data')
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd

# This script assumes 'appraisal_scope_df' is your prepared DataFrame.
# Select only numeric columns for the correlation matrix
numeric_df = appraisal_scope_df.select_dtypes(include=['float64', 'int64'])

# Calculate the Spearman correlation with the target variable
spearman_corr = numeric_df.corr(method='spearman')['total_cases_opened'].sort_values(ascending=False)

print("--- Spearman's Rank Correlation with 'total_cases_opened' ---")
print(spearman_corr)

**(Original) Code Cell:**

In [ ]:
import pandas as pd

# This script assumes 'appraisal_scope_df' is your prepared DataFrame.
# Select only numeric columns for the correlation matrix
numeric_df = appraisal_scope_df.select_dtypes(include=['float64', 'int64'])

# Calculate the Kendall's Tau correlation with the target variable
kendall_corr = numeric_df.corr(method='kendall')['total_cases_opened'].sort_values(ascending=False)

print("--- Kendall's Tau Correlation with 'total_cases_opened' ---")
print(kendall_corr)

**(Original) Code Cell:**

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# --- 1. Prepare the Data ---
# This script assumes 'appraisal_scope_df' is your prepared DataFrame.
# The seasonal_decompose function requires a DatetimeIndex.
if not isinstance(appraisal_scope_df.index, pd.DatetimeIndex):
    appraisal_scope_df['date'] = pd.to_datetime(appraisal_scope_df['date'])
    appraisal_scope_df = appraisal_scope_df.set_index('date')


# --- 2. Perform and Plot the Decomposition ---
# We use a period of 12 because the data is monthly.
decomposition = seasonal_decompose(
    appraisal_scope_df['total_cases_opened'],
    model='additive',
    period=12
)

# The .plot() method automatically creates a figure with the four components
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.suptitle('Seasonal Decomposition of Total Cases Opened', y=1.02)
plt.tight_layout()
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# This script assumes 'appraisal_scope_df' is your prepared DataFrame.

# --- 1. Prepare the Data ---
# The adfuller function requires a series with no missing values.
target_series = appraisal_scope_df['total_cases_opened'].dropna()

# --- 2. Perform the ADF Test ---
result = adfuller(target_series)

# --- 3. Print and Interpret the Results ---
print('--- Augmented Dickey-Fuller Test Results ---')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print('Critical Values:')
for key, value in result[4].items():
    print(f'\t{key}: {value}')

print("\n--- Conclusion ---")
if result[1] <= 0.05:
    print("The p-value is less than 0.05, so we reject the null hypothesis.")
    print("Result: The data is likely stationary.")
else:
    print("The p-value is greater than 0.05, so we fail to reject the null hypothesis.")
    print("Result: The data is likely non-stationary.")

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import statsmodels.api as sm

# --- 1. Load and Prepare Data ---
df = pd.read_csv(r'C:\Users\jhgonzalez\Desktop\merged_forecasting_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()

# --- 2. Feature Engineering & Final Selection ---
appraisal_scope_df['month'] = appraisal_scope_df.index.month
features = ['TotalOrders', 'business_day_count'] # Using the final recommended features
target = 'total_cases_opened'
appraisal_scope_df = appraisal_scope_df.dropna(subset=[target] + features)

# --- 3. Split Data ---
X = appraisal_scope_df[features]
y = appraisal_scope_df[target]
split_point = len(appraisal_scope_df) - 6
X_train = X.iloc[:split_point]
y_train = y.iloc[:split_point]

# --- 4. Fit the OLS Model ---
X_train_sm = sm.add_constant(X_train)
model_sm = sm.OLS(y_train, X_train_sm).fit()
residuals = model_sm.resid

print("OLS model has been fitted. Residuals are ready for testing.")

**(Original) Code Cell:**

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

# Perform the Ljung-Box test on the model's residuals
ljung_box_results = acorr_ljungbox(residuals, lags=[12], return_df=True)

print("--- Ljung-Box Test for Autocorrelation in Residuals ---")
print(ljung_box_results)

**(Original) Code Cell:**

In [ ]:
from scipy.stats import shapiro

# Perform the Shapiro-Wilk test on the residuals
shapiro_test = shapiro(residuals)

print("\n--- Shapiro-Wilk Test for Normality of Residuals ---")
print(f"Test Statistic: {shapiro_test.statistic}")
print(f"P-value: {shapiro_test.pvalue}")

**(Original) Code Cell:**

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# The test requires a DataFrame with the two time series
causality_df = appraisal_scope_df[['total_cases_opened', 'TotalOrders']].dropna()

# Perform the test. We'll test up to a lag of 3 months.
granger_results = grangercausalitytests(causality_df, maxlag=3, verbose=True)

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# --- Function to Calculate SMAPE ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

# --- 1. Load and Prepare Data ---
df = pd.read_csv(r'C:\Users\jhgonzalez\Desktop\merged_forecasting_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()

# --- 2. Feature Engineering ---
# Create lagged features based on Granger Causality results
appraisal_scope_df['TotalOrders_lag_2'] = appraisal_scope_df['TotalOrders'].shift(2)
appraisal_scope_df['TotalOrders_lag_3'] = appraisal_scope_df['TotalOrders'].shift(3)

# Define features and target, and drop rows with missing values from lagging
features = ['business_day_count', 'TotalOrders_lag_2', 'TotalOrders_lag_3']
target = 'total_cases_opened'
appraisal_scope_df = appraisal_scope_df.dropna(subset=[target] + features)

# --- 3. Select Features and Split Data ---
X = appraisal_scope_df[features]
y = appraisal_scope_df[target]

# Split the data to test on the last 6 months
split_point = len(appraisal_scope_df) - 6
X_train = X.iloc[:split_point]
y_train = y.iloc[:split_point]
X_test = X.iloc[split_point:]
y_test = y.iloc[split_point:]

# --- 4. Build and Train the SARIMAX Model ---
# The (p,d,q) and (P,D,Q,s) orders are hyperparameters.
# These are common starting points for monthly, seasonal data.
model = SARIMAX(y_train,
                exog=X_train,
                order=(1, 1, 1),
                seasonal_order=(1, 1, 0, 12))

results = model.fit(disp=False)
print(results.summary())


# --- 5. Make Predictions and Evaluate ---
predictions = results.get_forecast(steps=len(X_test), exog=X_test)
predicted_mean = predictions.predicted_mean

mae = mean_absolute_error(y_test, predicted_mean)
smape_error = smape(y_test, predicted_mean)

print("\n--- SARIMAX Model Evaluation (Last 6 Months) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {smape_error:.2f}%")


# --- 6. Visualize the Results ---
plt.figure(figsize=(14, 7))
plt.plot(appraisal_scope_df.index, appraisal_scope_df[target], label='Actual Cases')
plt.plot(predicted_mean.index, predicted_mean.values, label='SARIMAX Forecast', linestyle='--')
plt.title('Actual vs. SARIMAX Forecast for Appraisal Scope')
plt.xlabel('Date')
plt.ylabel('Total Cases Opened')
plt.legend()
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- Function to Calculate SMAPE ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

# --- 1. Load and Prepare Data ---
df = pd.read_csv(r'C:\Users\jhgonzalez\Desktop\merged_forecasting_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()

# --- 2. Feature Engineering ---
appraisal_scope_df['month'] = appraisal_scope_df.index.month
features = ['TotalOrders', 'business_day_count', 'month']
target = 'total_cases_opened'
appraisal_scope_df = appraisal_scope_df.dropna(subset=[target] + features)

# --- 3. Select Features and Split Data ---
X = appraisal_scope_df[features]
y = appraisal_scope_df[target]

# Split the data to test on the last 6 months
split_point = len(appraisal_scope_df) - 6
X_train = X.iloc[:split_point]
y_train = y.iloc[:split_point]
X_test = X.iloc[split_point:]
y_test = y.iloc[split_point:]

# --- 4. Build, Train, and Evaluate ---
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
smape_error = smape(y_test, predictions)

print("--- OLS Model Evaluation (Last 6 Months) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {smape_error:.2f}%")

# --- 5. Generate Detailed Statistical Summary ---
X_train_sm = sm.add_constant(X_train)
model_sm = sm.OLS(y_train, X_train_sm).fit()
print("\n--- OLS Regression Statistical Summary ---")
print(model_sm.summary())

# --- 6. Visualize the Results ---
plt.figure(figsize=(14, 7))
plt.plot(appraisal_scope_df.index, appraisal_scope_df[target], label='Actual Cases')
plt.plot(X_test.index, predictions, label='OLS Forecast', linestyle='--')
plt.title('Actual vs. OLS Forecast for Appraisal Scope (Last 6 Months Test)')
plt.xlabel('Date')
plt.ylabel('Total Cases Opened')
plt.legend()
plt.grid(True)
plt.show()

**(Original) Code Cell:**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import statsmodels.api as sm

# --- Function to Calculate SMAPE ---
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

# --- 1. Load and Prepare Data ---
df = pd.read_csv(r'C:\Users\jhgonzalez\Desktop\merged_forecasting_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
appraisal_scope_df = df[df['client'] == 'Appraisal Scope'].copy()
target = 'total_cases_opened'

# --- 2. Feature Engineering ---
# Use the [1, 12] lag combination for momentum and seasonality
df_lagged = appraisal_scope_df.copy()
df_lagged['lag_1'] = df_lagged[target].shift(1)
df_lagged['lag_12'] = df_lagged[target].shift(12)
df_lagged = df_lagged.dropna()

# --- 3. Select Features and Split Data ---
features = ['lag_1', 'lag_12']
X = df_lagged[features]
y = df_lagged[target]

# Split the data to test on the last 6 months
split_point = len(df_lagged) - 6
X_train = X.iloc[:split_point]
y_train = y.iloc[:split_point]
X_test = X.iloc[split_point:]
y_test = y.iloc[split_point:]

# --- 4. Build, Train, and Evaluate ---
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
smape_error = smape(y_test, predictions)

print("--- Weighted Lag Average Model Evaluation (Last 6 Months) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Symmetric Mean Absolute Percentage Error (SMAPE): {smape_error:.2f}%")

# --- 5. Generate Detailed Statistical Summary ---
X_train_sm = sm.add_constant(X_train)
model_sm = sm.OLS(y_train, X_train_sm).fit()
print("\n--- OLS Regression Statistical Summary ---")
print(model_sm.summary())

# --- 6. Visualize the Results ---
plt.figure(figsize=(14, 7))
plt.plot(df_lagged.index, df_lagged[target], label='Actual Cases')
plt.plot(X_test.index, predictions, label='Forecast', linestyle='--')
plt.title('Actual vs. Forecast (Weighted Lag Average Model)')
plt.xlabel('Date')
plt.ylabel('Total Cases Opened')
plt.legend()
plt.grid(True)
plt.show()

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**

**(Original) Code Cell:**